# Robust Equal Sum partitioning using DQM

We have a list of N values A. The goal is to find two sets of variable whose sum is equal, enabling outliers.

## ExactDQM

In [22]:
from dimod import DiscreteQuadraticModel, ExactDQMSolver
from itertools import combinations
import numpy as np

In [23]:
values = [7, 2, 3, 1, 8, 3, 1, 2, 10]
print(np.sum(values))  # no good partition, as sum is uneven

dqm = DiscreteQuadraticModel()
n = len(values)
m = 3 # num_partitions

x = {i: dqm.add_variable(m) for i in range(n)}
# x0 = {0->neither, 1->set1, 2->set2}

37


In [24]:
dqm.add_linear_equality_constraint(
[(x[i], 1, values[i]) for i in range(n)] + [(x[i], 2, -values[i]) for i in range(n)],
constant=0,
lagrange_multiplier=100)

In [25]:
for i in range(n):
    dqm.set_linear_case(x[i],0,1)

In [26]:
res = ExactDQMSolver().sample_dqm(dqm).truncate(5)
print(res)

  0 1 2 3 4 5 6 7 8 energy num_oc.
0 1 2 2 0 2 2 1 2 1    1.0       1
1 1 2 1 2 1 2 0 2 2    1.0       1
2 0 2 1 2 2 2 2 1 1    1.0       1
3 1 2 2 2 2 0 2 2 1    1.0       1
4 1 2 0 2 1 2 2 1 2    1.0       1
['INTEGER', 5 rows, 5 samples, 9 variables]


In [27]:
sample = res.first.sample

print(sample)
print(sum(values))
for k in range(m):
    set1 = [values[i] for i in x if sample[x[i]] == k]
    print(sum(set1), set1)

{0: 1, 1: 2, 2: 2, 3: 0, 4: 2, 5: 2, 6: 1, 7: 2, 8: 1}
37
1 [1]
18 [7, 1, 10]
18 [2, 3, 8, 3, 2]


In [28]:
samplebis = res.record[1].sample

print(samplebis)
print(sum(values))
for k in range(m):
    set1 = [values[i] for i in x if samplebis[x[i]] == k]
    print(sum(set1), set1)

[1 2 1 2 1 2 0 2 2]
37
1 [1]
18 [7, 3, 8]
18 [2, 1, 3, 2, 10]


# On hardware

In [10]:
# from dwave.system import DWaveSampler, EmbeddingComposite

In [ ]:
# sampler = EmbeddingComposite(DWaveSampler())
# response = sampler.sample_qubo(dqm,
#                                num_reads=1000,
#                                label='Partitioning')

In [29]:
from dwave.system import LeapHybridDQMSampler

In [31]:
# sampler = EmbeddingComposite(LeapHybridDQMSampler())
sampler = LeapHybridDQMSampler()

In [32]:
response = sampler.sample_dqm(dqm, label='Example - Graph Partitioning DQM')

In [ ]:
print(response.first.sample)